In [ ]:
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
import pathlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import os
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

In [ ]:
data_dir = pathlib.Path(r"D:\BT\2433_Emerging_Technologies\data\Set_A\Set_A\Train")

In [ ]:
listImages = list(data_dir.glob('*/*.jpg')) + list(data_dir.glob('*/*.png')) + list(data_dir.glob('*/*.jpeg')) + list(data_dir.glob('*/*.jfif')) 

image_count = len(listImages)
print(image_count)
listImages

In [ ]:
batch_size = 64
img_height = 320
img_width = 320

In [ ]:
def square_fn(x):
    return x ** 2

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names 
print(class_names)

In [ ]:
data_augmentation = tf.keras.Sequential([
    # layers.RandomFlip("horizontal_and_vertical"),
    # layers.RandomRotation(0.2),
    # layers.RandomZoom(0.2),
    # layers.RandomContrast(0.2),
    # layers.RandomBrightness(factor=0.2),
    # layers.RandomTranslation(0.1, 0.1),
    # layers.GaussianNoise(0.1)
    layers.RandomFlip("horizontal"),
    layers.RandomBrightness(0.05),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.05, 0.05),
])

In [ ]:
num_classes = len(class_names)

# khai báo các Layers
def create_model():
  base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                             include_top=False,
                             weights='imagenet')
  base_model.trainable = True

  fine_tune_at = 50
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
  model = Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    # layers.Conv2D(16, 3, padding='same', activation='relu'),
    # layers.MaxPooling2D(),
    # layers.Conv2D(32, 3, padding='same', activation='relu'),
    # layers.MaxPooling2D(),
    # layers.Conv2D(64, 3, padding='same', activation='relu'),
    # layers.MaxPooling2D(),
    # layers.Flatten(),
    # layers.Dense(128, activation='relu'),
    # layers.Dropout(0.5),
    # layers.Dense(num_classes, activation='softmax')
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
  ]) 

  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-5,
        decay_steps=10000,
        decay_rate=0.96,
        staircase=True
    )
  model.compile(
    # optimizer='adam',
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']) 
  return model

In [ ]:
for class_name in class_names:
    count = len(list((data_dir / class_name).glob("*")))
    print(f"{class_name}: {count} images")

In [ ]:
class StopTrainingAtAccuracy(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') is not None and logs.get('accuracy') >= 0.98:
            print(f"\n Dừng huấn luyện: Độ chính xác đạt {logs.get('accuracy')}")
            self.model.stop_training = True 
stop_callback = StopTrainingAtAccuracy()
# steps_per_epoch = len(train_ds)
validation_steps = len(val_ds)

In [ ]:
plt.figure(figsize=(10, 10))
train_ds = train_ds.apply(tf.data.experimental.ignore_errors())
for images, labels in train_ds.take(1):  # Dữ liệu gốc chưa được chuẩn hóa
    for i in range(12):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
for images, labels in train_ds.take(1):
    plt.imshow(images[0].numpy().astype("uint8"))
    plt.title(f"Label: {class_names[labels[0]]}")
    plt.show()

In [ ]:
best_model_path = "best_model_so_far.keras"
best_val_acc_so_far = 0.0
best_model = None

# Nếu đã có model lưu từ trước, thì tải nó
if os.path.exists(best_model_path):
    print("Tải lại model tốt nhất từ trước...")
    best_model = load_model(best_model_path)
    # Cần đo lại độ chính xác validation của model này
    loss, acc = best_model.evaluate(val_ds, verbose=0)
    best_val_acc_so_far = acc
    print(f"Val Accuracy hiện tại của model đã lưu: {acc:.4f}")

In [ ]:
y_train = np.concatenate([y.numpy() for x, y in train_ds])
class_weights = compute_class_weight(
      class_weight="balanced",
      classes=np.unique(y_train),
      y=y_train
  )
class_weights = dict(enumerate(class_weights))

In [ ]:
attempt = 1
best_val_acc = best_val_acc_so_far
best_model_path = "best_model_so_far.keras"

while True:
    model = create_model()
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=500,
        class_weight=class_weights,
        #không cần thiết
        # steps_per_epoch=steps_per_epoch,
        # validation_steps=validation_steps,
        callbacks=[early_stop, tensorboard_callback, stop_callback],
        verbose=1
    )
    acc = history.history['accuracy'][-1]
    val_acc = history.history['val_accuracy'][-1]

    print(f"Accuracy: {acc:.4f}, Val Accuracy: {val_acc:.4f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        print(f"Lưu model mới tốt hơn với Val Accuracy = {val_acc:.4f}")
        model.save(best_model_path)
    
    if acc >= 1.0 and 0.8 <= val_acc <= 1.0:
        print("Đạt yêu cầu. Lưu model...")
        model.save("model_accepted.keras")
        break
    else:
        print("Chưa đạt yêu cầu. Huấn luyện lại...\n")
        attempt += 1


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend()
plt.show()

In [ ]:
print(train_ds.class_names)

In [ ]:
best_model.summary()

In [ ]:
best_model_path = "best_model_so_far.keras"
best_val_acc_so_far = 0.0
best_model = None

# Nếu đã có model lưu từ trước, thì tải nó
if os.path.exists(best_model_path):
    print("Tải lại model tốt nhất từ trước...")
    best_model = load_model(best_model_path, custom_objects={"preprocess_input": preprocess_input})
    # Cần đo lại độ chính xác validation của model này
    loss, acc = best_model.evaluate(val_ds, verbose=0)
    best_val_acc_so_far = acc
    print(f"Val Accuracy hiện tại của model đã lưu: {acc:.4f}")

In [ ]:
print(len(class_names))

In [ ]:
y_true = []  # True labels
y_pred = []  # Predicted labels

for images, labels in train_ds:
    predictions = best_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

In [ ]:
pred = best_model.predict(np.expand_dims(img, axis=0))
for i, prob in enumerate(pred[0]):
    print(f"{i}: {class_names[i]} - {prob * 100:.2f}%")

In [ ]:
test_dir = pathlib.Path(r"D:\\BT\\2433_Emerging_Technologies\\data\\Set_A\\Set_A\\Test") 
listImages_test = list(test_dir.glob('*.jpg')) + list(test_dir.glob('*.webp')) + list(test_dir.glob('*.png')) + list(test_dir.glob('*.jpeg')) + list(test_dir.glob('*.jfif'))
plt.figure(figsize=(20, 20))
for i in range(len(listImages_test)):
    img = tf.keras.utils.load_img(
        listImages_test[i], target_size=(img_width, img_height)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = best_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    predicted_name = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)
    true_name = listImages_test[i].name  # Extract the true name from the file name
    print(i, f"Predict {predicted_name} with a {confidence:.2f} percent confidence. ::REAL {true_name}")
    plt.subplot((len(listImages_test)//6) + 1, 6, i + 1)
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_name}\nConfidence: {confidence:.2f}%\nTrue: {true_name}")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
predictions = best_model.predict(images)
confidence_scores = tf.reduce_max(predictions, axis=1)  # Lấy giá trị lớn nhất
print("Confidence Scores:", confidence_scores.numpy())

In [ ]:
predicted_class = np.argmax(predictions[0])
print(f"Predict {class_names[predicted_class]} with a {confidence:.2f}% confidence.")
for i in range(len(images)):
    predicted_class = np.argmax(predictions[i])
    confidence = 100 * predictions[i][predicted_class]
    print(f"{i} Predict {class_names[predicted_class]} with a {confidence:.2f}% confidence.")

11
10
10
13
13
13